<a href="https://colab.research.google.com/github/VictoryBeforeFight/KOSA_ML_Project/blob/main/final_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 웹 광고 클릭률 데이터 (출처 : 데이콘)
- 시간 순으로 나열된 7일 동안의 웹 광고 클릭 로그
- ID: train 데이터 샘플 고유 ID
- Click: 예측 목표인 클릭 여부
- 0: 클릭하지 않음, 1: 클릭
- F01 ~ F39 : 각 클릭 로그와 연관된 Feature
- 개인정보 보호를 위해 상세 정보는 비식별 처리됨

## 데이터 로드 및 라이브러리 임포트

### 데이터 로드

In [ ]:
# 데이터프레임 형식으로 데이터 로드
df = pd.read_parquet("/content/drive/MyDrive/KOSA_데이터분석 팀 프로젝트/Pilot Project/CTR/train.parquet")

### 라이브러리 임포트

In [ ]:
!pip install optuna

In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import optuna

## 2차 데이터 전처리

### 결측치 처리
- 연속형 변수의 결측치를 중앙값으로 대체
- 범주형 변수의 결측치를 최빈값인 0으로 대체
- 결측치가 많았던 F11, F22, F29 열은 제거

In [ ]:
# 연속형 변수 결측치를 중앙값으로 대체
def fill_missing_values_with_median(df):
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())
    return df

def fill_categorical_missing_with_negative_one(df):
    # 범주형 변수 선별
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns

    # 범주형 변수의 결측치를 -1로 대체
    df[categorical_cols] = df[categorical_cols].fillna(-1)

    return df

# 연속형 변수 결측치를 0으로 대체
def fill_missing_values_with_zero(df):
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    df[numeric_cols] = df[numeric_cols].fillna(0)
    return df

In [ ]:
df = fill_missing_values_with_median(df)
df = fill_categorical_missing_with_negative_one(df)

### 범주형 데이터 인코딩
- 범주형 데이터는 머신러닝 모델의 학습이나 평가를 위해 사용할 수 없기 때문에 인코딩하여 사용

In [ ]:
from sklearn.preprocessing import LabelEncoder

def categorical_label(df) :
    # 범주형 데이터 뽑기
    categorical_cols = df.select_dtypes(include = ['object']).columns

    #타입변경
    df[categorical_cols] = df[categorical_cols].astype(str)

    for cols in categorical_cols :
        #label encoder 불러오기
        le = LabelEncoder()
        # 범주형 -> 수치형 데이터 변환
        df[cols] = le.fit_transform(df[cols])

In [ ]:
df = categorical_label(df)

### 학습용 데이터와 평가용 데이터 분리

In [ ]:
X = df.drop(columns='Click')
y = df.loc[:, 'Click']

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 데이터 샘플링
- 데이터 양이 많고, 종속 변수의 비율이 8.5 : 1.5 분포되어 있어 정확한 학습과 예측이 어려울 것이라고 판단하여 언더 샘플링으로 진행

In [ ]:
def random_undersampling(X, y):
    """
    언더샘플링을 수행하여 균형 잡힌 데이터셋을 생성합니다.

    Args:
        X (pandas.DataFrame): 입력 데이터
        y (pandas.Series): 레이블 데이터

    Returns:
        tuple: 균형 잡힌 X, y 데이터셋
    """

    X.index = y.index

    positive_examples = X[y == 1]
    negative_examples = X[y == 0]
    num_positive_samples = len(positive_examples)

    random_negative_samples = negative_examples.sample(n=num_positive_samples, random_state=42)

    balanced_X = pd.concat([positive_examples, random_negative_samples])
    balanced_y = pd.concat([pd.Series([1] * len(positive_examples)),
                            pd.Series([0] * len(random_negative_samples))])

    return balanced_X, balanced_y

In [ ]:
balanced_X, balanced_y = random_undersampling(X_train, y_train)

## 2차 모델링


### 평가 지표를 텍스트로 출력하는 함수 정의

In [ ]:
def evaluate_model(model, X_test, y_test):
    """
    모델 성능 평가 함수
    """
    # AUC 계산
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)
    print(f'AUC: {auc:.4f}')

    # 성능 평가 보고서 생성
    report = classification_report(y_test, model.predict(X_test))
    print(report)

    # 혼동 행렬 출력
    cm = confusion_matrix(y_test, model.predict(X_test))
    print(cm)

In [ ]:
def plot_feature_importance(model, X_train):
    """
    특성 중요도 시각화 함수
    """
    feature_importances = pd.DataFrame({
        'feature': X_train.columns,
        'importance': model.feature_importances_
    }).sort_values(by='importance', ascending=False)

    plt.figure(figsize=(10, 8))
    sns.barplot(x='importance', y='feature', data=feature_importances)
    plt.title('Feature Importance')
    plt.show()

### 평가 지표를 그래프로 출력하는 함수 정의

In [ ]:
def evaluate_model(model, X_test, y_test, model_name):
    # 예측값 생성
    y_pred = model.predict(X_test)

    # 혼동 행렬 시각화
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted Target')
    plt.ylabel('Actual Target')
    plt.title(f'{model_name} Confusion Matrix')
    plt.show()

    # ROC 곡선 시각화
    recall = recall_score(y_test, y_pred)
    fallout = 1 - recall_score(y_test, y_pred, pos_label=0)
    fpr, tpr, _ = roc_curve(y_test, model.decision_function(X_test))

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=model_name)
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
    plt.plot([fallout], [recall], 'ro', ms=10)
    plt.xlabel('False Positive Rate (Fall-Out)')
    plt.ylabel('True Positive Rate (Recall)')
    plt.title(f'{model_name} ROC Curve')
    plt.legend()
    plt.show()

### Logistc Regression

In [ ]:
# 회귀분류 모델 학습
lr_model = LogisticRegression()
lr_model.fit(balanced_X, balanced_y)

# 테스트 데이터로 예측
y_pred = lr_model.predict(X_test)

In [ ]:
evaluate_model(dt_model, X_test, y_test)

### Decision Tree

In [ ]:
# 의사결정나무 모델 학습
dt_model = DecisionTreeClassifier()
dt_model.fit(balanced_X, balanced_y)

# 테스트 데이터로 예측
y_pred = dt_model.predict(X_test)

In [ ]:
evaluate_model(dt_model, X_test, y_test)

### Random Forest

In [ ]:
# 랜덤포레스트 모델 학습
rf_model = RandomForestClassifier()
rf_model.fit(balanced_X, balanced_y)

# 테스트 데이터로 예측
y_pred = rf_model.predict(X_test)

In [ ]:
evaluate_model(rf_model, X_test, y_test)

### XGBoost

In [ ]:
# XGBoost모델 학습
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = xgb_model.predict(X_test)

In [ ]:
evaluate_model(xgb_model, X_test, y_test)

### LGBM

In [ ]:
# 모델 학습
lgb_model = LGBMClassifier()
lgb_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = lgb_model.predict(X_test)

In [ ]:
evaluate_model(lgb_model, X_test, y_test)

### CatBoost

In [ ]:
# 모델 학습
cb_model = CatBoostClassifier()
cb_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = cb_model.predict(X_test)

In [ ]:
evaluate_model(cb_model, X_test, y_test)

## 하이퍼 파라미터 튜닝

### CatBoost Tuning

In [ ]:
OPTUNA_OPTIMIZATION = True

def objective(trial):
    # train_x, valid_x, train_y, valid_y = train_test_split(x_train,y_train, test_size=0.3)

    #define parameters
    params = {
        'iterations':trial.suggest_int("iterations", 500, 3000),
        'objective':trial.suggest_categorical('objective',['CrossEntropy','Logloss']),
        'bootstrap_type':trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
        'od_wait':trial.suggest_int('od_wait', 500, 1000),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01,1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'random_strength': trial.suggest_uniform('random_strength',20,50),
        'depth': trial.suggest_int('depth',1,15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,20),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'verbose': False,
        "eval_metric":'AUC', # AUC로 성능 측정
        'task_type' : 'GPU',
    }

    if params['bootstrap_type'] == 'Bayesian':
        params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
    elif params['bootstrap_type'] == 'Bernoulli':
        params['subsample'] = trial.suggest_float('subsample', 0.1, 1)

    # model fit
    model = CatBoostClassifier(**params)
    model.fit(
        train_X, train_y, eval_set=[(test_X, test_y)],
        use_best_model=True
    )

    # validation prediction

    y_pred = model.predict_proba(test_X)
    y_score=roc_auc_score(test_y,y_pred[:, 1])
    return y_score

In [ ]:
import optuna
study = optuna.create_study(
    direction='maximize',
    study_name='CatbClf'
)

study.optimize(
    objective,
    n_trials=20
)

In [ ]:
Best_params = study.best_trial.params
print(f"Best Trial: {study.best_trial.value}")
print(f"Best Params: {study.best_trial.params}")

### RandomForest Tuning

In [ ]:
OPTUNA_OPTIMIZATION = True

def objective(trial):
    # train_x, valid_x, train_y, valid_y = train_test_split(x_train,y_train, test_size=0.3)

    #define parameters
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 5000),
        'max_depth' : trial.suggest_int('max_depth', 3, 10),
        'max_features' : trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf' : trial.suggest_int('min_samples_leaf', 1, 10),
        'bootstrap' : trial.suggest_categorical('bootstrap', [True, False]),
        'random_state': 42,
        'n_streams':1  # 재현성을 위해 n_streams를 1로 설정
    }


    # model fit
    model = RandomForestClassifier(**param)
    model.fit(train_X, train_y)

    # validation prediction

    y_pred = model.predict_proba(test_X)
    y_score=roc_auc_score(test_y,y_pred[1])
    return y_score

In [ ]:
import optuna
study = optuna.create_study(
    direction='maximize',
    study_name='RFClf'
)

study.optimize(
    n_trials=20,
    timeout=5400
)

In [ ]:
Best_params = study.best_trial.params
print(f"Best Trial: {study.best_trial.value}")
print(f"Best Params: {study.best_trial.params}")

### XGBoost Tuning

In [ ]:
# 목적 함수 정의
def objective(trial):
    # 튜닝할 하이퍼파라미터 범위 정의
    max_depth = trial.suggest_int('max_depth', 2, 10)
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)

    # xgboost 모델 생성 및 학습
    model = xgb.XGBClassifier(
        max_depth=max_depth,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        random_state=42,
        tree_method="gpu_hist"
        # task_type="GPU"
    )
    model.fit(X_train, y_train)

    # 모델 평가 (목적 함수)
    score = model.score(X_test, y_test)
    return score

# 옵튜나 study 객체 생성 및 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# 최적의 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)
print('Best score: ', study.best_value)

## 데이터 탐색

## 1차 데이터 전처리

## 1차 모델링